In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
import warnings
import pandas as pd

warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

import warnings

warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')


lista_produtos = pd.read_excel(r"C:\Users\roberto.monteiro\Downloads\base_produtos.xlsx")

# Abrir ChromeDriver em segundo plano
options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-extensions")
options.add_argument("--use-fake-ui-for-media-stream")
options.add_argument("--js-flags=--harmony")
options.add_argument('--disable-logging')
options.add_argument('log-level=0')
options.add_argument('log-level=1')
options.add_argument('log-level=2')
options.add_argument('log-level=3')
options.add_experimental_option('excludeSwitches', ['enable-logging'])
chrome_prefs = {"download.default_directory": r"C:\Users\roberto.monteiro\Downloads\lista1"}
options.experimental_options["prefs"] = chrome_prefs
navegador = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)


def pesquisa_google(navegador, produto, preco_min, preco_max, termos):    
    lista_ofertas = []
    navegador.get('https://www.google.com/')
    navegador.find_element(By.CLASS_NAME, 'gLFyf').send_keys(produto, Keys.ENTER)

    navegador.find_element(By.LINK_TEXT, 'Shopping').click()

    resultados = navegador.find_elements(By.CLASS_NAME, 'i0X6df')

    for resultado in resultados:
        nome  = resultado.find_element(By.CLASS_NAME,'tAxDx').text
        preco  = resultado.find_element(By.CLASS_NAME,'a8Pemb').text
        link = resultado.find_element(By.TAG_NAME,'a').get_attribute('href')

        if nome != '' and preco != '' and link != '':
            status = True
        for tb in produto.split(' '):
            if tb.lower() not in nome.lower():
                status = False
                
        for tb in str(termos).split(' '):
            if tb.lower() in nome.lower():
                status = False



        if status == True and preco_max > float(preco[3:].replace('.','').replace(',','.')) > preco_min:                
            #print(f'Nome: {nome}, Preço: {preco} e Link: {link}')
            dados = f'{nome},;,{preco},;,{link},;,google_shopping'
            if dados not in lista_ofertas:
                lista_ofertas.append(dados)
    return lista_ofertas


def pesquisa_buscape(navegador, produto, preco_min, preco_max, termos):
    lista_ofertas = []
    navegador.get('https://www.buscape.com.br/')

    navegador.find_element(By.TAG_NAME, 'input').send_keys(produto, Keys.ENTER)
    time.sleep(5)

    resultados = navegador.find_elements(By.CLASS_NAME, 'SearchCard_ProductCard_Inner__7JhKb')

    for resultado in resultados:
        nome   = resultado.find_element(By.TAG_NAME,'h2').text
        preco  = resultado.find_element(By.TAG_NAME,'p').text
        link   = resultado.get_attribute('href')



        if nome != '' and preco != '' and link != '':
            status = True
        for tb in produto[0].split(' '):
            if tb.lower() not in nome.lower():
                status = False

        for tb in str(termos).split(' '):
            if tb.lower() in nome.lower():
                status = False



        if status == True and preco_max > float(preco[3:].replace('.','').replace(',','.')) > preco_min:                
            #print(f'Nome: {nome}, Preço: {preco} e Link: {link}')
            dados = f'{nome},;,{preco},;,{link},;,buscape'
            if dados not in lista_ofertas:

                lista_ofertas.append(dados)
    return lista_ofertas


listas = []
for tb in lista_produtos.values:
    lista = pesquisa_google(navegador, tb[0], tb[1], tb[2], tb[3])
    lista1 = pesquisa_buscape(navegador, tb[0], tb[1], tb[2], tb[3])
    listas += lista + lista1
    
navegador.quit()

tabela = []
for tb in listas:
    tabela.append(tb.split(',;,'))
    
tabela = pd.DataFrame(tabela,columns=['Nome', 'Preco', 'Link', 'Site'])
tabela



,Nome,Preco,Link,Site
0,Smartphone Galaxy S22 5G 128 GB 8gb Ram Preto ...,"R$ 3.599,10",https://www.google.com/shopping/product/146592...,google_shopping
1,Samsung Galaxy S22 5G - 128 GB - 8gb Ram - Vio...,"R$ 3.365,00",https://www.google.com/shopping/product/244216...,google_shopping
2,Usado: Samsung Galaxy S22 5GB 128GB Preto Muit...,"R$ 2.528,10",https://www.google.com/url?url=https://www.mag...,google_shopping
3,Usado: Samsung Galaxy S22 5GB 128GB Preto Exce...,"R$ 2.690,10",https://www.google.com/url?url=https://www.mag...,google_shopping
4,Smartphone Samsung Galaxy S22 5G 128Gb 8Gb Ram...,"R$ 3.599,00",https://www.google.com/url?url=https://www.car...,google_shopping
...,...,...,...,...
62,Iphone 12 Mini 64 Gb | Celular Apple | Usado,"R$ 3.665,00",https://www.google.com/url?url=https://www.enj...,google_shopping
63,Smartphone Apple iPhone 12 64GB Câmera Dupla,"R$ 3.599,00",https://www.buscape.com.br/celular/smartphone-...,buscape
64,Smartphone Apple iPhone 12 Mini 64GB Câmera Dupla,"R$ 3.779,10",https://www.buscape.com.br/celular/smartphone-...,buscape
65,Smartphone Apple iPhone 12 Vermelho 64GB Câmer...,"R$ 3.779,10",https://www.buscape.com.br/celular/smartphone-...,buscape


In [14]:
import smtplib
import email.message

def enviar_email():  
    corpo_email = """
    <p>Bom dia</p>
    <p>Roberto</p>
    """

    msg = email.message.Message()
    msg['Subject'] = "Teste"
    msg['From'] = 'assessorobertomonteiro@gmail.com'
    msg['To'] = 'roberto.monteiro@loreal.com'
    password = 'iurd5331' 
    msg.add_header('Content-Type', 'text/html')
    msg.set_payload(corpo_email )

    s = smtplib.SMTP('smtp.gmail.com: 587')
    s.starttls()
    # Login Credentials for sending the mail
    s.login(msg['From'], password)
    s.sendmail(msg['From'], [msg['To']], msg.as_string().encode('utf-8'))
    print('Email enviado')

enviar_email()

SMTPAuthenticationError: (534, b'5.7.9 Application-specific password required. Learn more at\n5.7.9  https://support.google.com/mail/?p=InvalidSecondFactor v2-20020a0568301bc200b0068bb73bd95esm8703423ota.58 - gsmtp')

In [8]:
pip install pywin32

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install ruamel_yaml==0.11.14

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [12 lines of output]
  Traceback (most recent call last):
    File "<string>", line 2, in <module>
    File "<pip-setuptools-caller>", line 34, in <module>
    File "C:\Users\roberto.monteiro\AppData\Local\Temp\pip-install-zq4wtbn8\ruamel-yaml_02af438d5a2c4052abb582f7febcbbeb\setup.py", line 165, in <module>
      pkg_data = _package_data(__file__.replace('setup.py', '__init__.py'))
    File "C:\Users\roberto.monteiro\AppData\Local\Temp\pip-install-zq4wtbn8\ruamel-yaml_02af438d5a2c4052abb582f7febcbbeb\setup.py", line 144, in _package_data
      data = literal_eval(u''.join(lines))
    File "C:\Users\roberto.monteiro\AppData\Local\Temp\pip-install-zq4wtbn8\ruamel-yaml_02af438d5a2c4052abb582f7febcbbeb\setup.py", line 116, in literal_eval
      return _convert(node_or_string)
    File "C:\Users\roberto.monteiro\AppData\Local\Temp\pip-install-zq4wtbn8\ruamel-yaml_02af438d5a2c40


     ------------------------------------ 223.3/223.3 kB 649.5 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
